In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import json
# data_json = json.load(open('/content/drive/MyDrive/train-v1.1.json','r'))

In [ ]:
!cp /content/drive/MyDrive/train_newqa_0.zip ./
!cp /content/drive/MyDrive/train_newqa_1.zip ./
!unzip /content/train_newqa_0.zip
!unzip /content/train_newqa_1.zip

Archive:  /content/train_newqa_0.zip
  inflating: train_newqa_4.json      
  inflating: train_newqa_0.json      
  inflating: train_newqa_1.json      
  inflating: train_newqa_3.json      
  inflating: train_newqa_2.json      
Archive:  /content/train_newqa_1.zip
  inflating: train_newqa_7.json      
  inflating: train_newqa_5.json      
  inflating: train_newqa_6.json      


In [ ]:
import json
data_json = {
    "version": '1.1',
    'data': []
}
data_json_1 = json.load(open('/content/train_newqa_0.json', 'r'))
data_json_2 = json.load(open('/content/train_newqa_1.json', 'r'))
data_json_3 = json.load(open('/content/train_newqa_2.json', 'r'))
data_json_4 = json.load(open('/content/train_newqa_3.json', 'r'))
data_json_5 = json.load(open('/content/train_newqa_4.json', 'r'))
data_json_6 = json.load(open('/content/train_newqa_5.json', 'r'))
data_json_7 = json.load(open('/content/train_newqa_6.json', 'r'))
data_json_8 = json.load(open('/content/train_newqa_7.json', 'r'))

data_json['data'] = data_json_1['data'] + data_json_2['data'] + data_json_3['data'] + data_json_4['data'] + data_json_5['data'] + data_json_6['data'] + data_json_7['data'] + data_json_8['data']

In [ ]:
len(data_json['data'])

66090

#Cut by context

In [ ]:
filted_data = {
    "version": '1.1',
    'data': []
}

k = 0
v= 0
for idx,i in enumerate(data_json['data']):
  for paragraph in i['paragraphs']:
    if len(paragraph['context']) > 3500:
      v = v+1
      continue
    for qas in paragraph['qas']:
      for a in qas['answers']:
        if a['text'] == "":
          # print(idx)
          k =k +1
          continue
        else:
          filted_data['data'].append(i)

print(k)
print(v)

0
15657


In [ ]:
len(filted_data['data'])

40805

In [ ]:
with open('/content/train_newqa_total.json', 'w') as outfile:
    json.dump(filted_data, outfile)

In [ ]:
!cp /content/train_newqa_total.json /content/drive/MyDrive

In [ ]:
k=0
for data in data_json['data']:
  for paragraph in data['paragraphs']:
    for qa in paragraph['qas']:
      for answer in qa['answers']:
        if answer['answer_start'] < 500:
          k=k+1

print(k)

38527


#Cut by answer start

In [ ]:
import random
import re

In [ ]:
random.seed(42)

In [ ]:
def find_random_dot(context, start, end):
  idx_dots = [m.start() + start for m in re.finditer(r"\.", context[start:])]
  idx_dots = [idx for idx in idx_dots if idx < 650 and idx > end]
  try:
    return random.choice(idx_dots)
  except:
    return -1

In [ ]:
ws_data = {
    "version": '1.1',
    'data': []
}

In [ ]:
k=0
v=0
len_context = 0
for data in data_json['data']:
  obj_1 = {
      "title": "",
      "paragraphs": []
  }
  for paragraph in data['paragraphs']:
    idx_dot=-1
    obj_2 = {
        'context' : "",
        'qas' : []
    }
    for qa in paragraph['qas']:
      obj_3 = {
          'question': qa['question'],
          'answers': [],
          'id': qa['id']
      }
      for answer in qa['answers']:
        obj_4 = {
            'answer_start':answer['answer_start'],
            'text': answer['text']
        }
        answer_end = answer['answer_start'] + len(answer['text']) + 1
        if answer['text'] == "":
            continue
        if answer['answer_start'] < 500:
          answer_start = answer['answer_start']
          idx_dot = find_random_dot(paragraph['context'], answer_start, answer_end)
          if idx_dot != -1:
            obj_3['answers'].append(obj_4)
          else:
            continue
        else:
          continue
      if len(obj_3['answers']) > 0:
        obj_2['qas'].append(obj_3)
    if idx_dot !=-1:
      obj_2['context'] = paragraph['context'][:idx_dot]
    else:
      continue
    if len(obj_2['qas']) > 0:
      obj_1['paragraphs'].append(obj_2)
  if len(obj_1['paragraphs']) >0:
    ws_data['data'].append(obj_1)
# print(k, ";",v)

In [ ]:
len(ws_data['data'])

36310

In [ ]:
file_name = '/content/train_newqa_ans500_ctx600.json'
with open(file_name, 'w') as outfile:
    json.dump(ws_data, outfile)

In [ ]:
!cp /content/train_newqa_ans500_ctx600.json /content/drive/MyDrive